In [96]:
import sys, os
sys.path.append('./modules')
from Monitoring_Tools import *
from OpenStack_Tools  import *

In [97]:
# TODO:
# ssh WRCO0 df -h / /opt/platform /etc/nova/instances
# Add new computes, add keys

In [98]:
from IPython.display import display
from IPython.display import HTML
import IPython.core.display as di

# This line will hide code by default when the notebook is exported as HTML
di.display_html('<script>jQuery(function() {if (jQuery("body.notebook_app").length == 0) { jQuery(".input_area").toggle(); jQuery(".prompt").toggle();}});</script>', raw=True)

# This line will add a button to toggle visibility of code blocks, for use with the HTML export version
di.display_html('''<button onclick="jQuery('.input_area').toggle(); jQuery('.prompt').toggle();">Toggle code</button>''', raw=True)

Toggle code

In [153]:
platform=os.getenv('OS_PLATFORM', 'poc2')
display_platform(platform)
inventory = read_inventory("$HOME/env/{}_hosts.ini".format(platform))
conn = connectToCloud(platform)
#inventory

In [100]:
show_notebook_url(platform, host_ip="10.3.216.210", port=8888)

#### Servers status (ping servers in 'ping_check' group)

In [101]:
display_html_ping_all(inventory, False)

ping(hos2[10.3.219.32]) ... ping(hos3[10.3.219.33]) ... ping(hos_vip[10.3.216.30]) ... ping(wrcl1[10.3.219.131]) ... ping(wrco1[10.3.219.178]) ... ping(hlm1[10.3.216.10]) ... ping(wrcl2[10.3.219.132]) ... ping(wrco3[10.3.219.134]) ... ping(hlm1vm[10.3.219.30]) ... ping(wrco0[10.3.219.177]) ... ping(wrcl_vip[10.3.219.130]) ... ping(hos1[10.3.219.31]) ... ping(wrco2[10.3.219.179]) ... 

wrco3[10.3.219.134],OK: 0.119 msec
wrcl2[10.3.219.132],OK: 0.137 msec
wrco0[10.3.219.177],OK: 0.113 msec
wrco1[10.3.219.178],OK: 0.110 msec
hos3[10.3.219.33],OK: 0.254 msec
wrco2[10.3.219.179],OK: 0.095 msec
hlm1vm[10.3.219.30],OK: 0.323 msec
hlm1[10.3.216.10],OK: 0.182 msec
hos1[10.3.219.31],OK: 0.211 msec
hos_vip[10.3.216.30],OK: 0.248 msec
wrcl_vip[10.3.219.130],OK: 0.132 msec


#### Show VMs (servers)

In [102]:
showServerList( conn )

2 flavors
11 images
0 servers


name,status,flavor,image,addresses


#### Show server port status

In [103]:
display_html_ping_ports_all(inventory)

ping(wrco5[10.3.219.135:22]) ... ping(hos2[10.3.219.32:22]) ... ping(hos3[10.3.219.33:22]) ... ping(hos_vip[10.3.216.30:22]) ... ping(wrcl1[10.3.219.131:22]) ... ping(wrco1[10.3.219.178:22]) ... ping(hlm1[10.3.216.10:22]) ... ping(wrcl2[10.3.219.132:22]) ... ping(wrco7[10.3.219.138:22]) ... ping(wrco3[10.3.219.134:22]) ... ping(hlm1vm[10.3.219.30:22]) ... ping(wrco0[10.3.219.177:22]) ... ping(wrco6[10.3.219.137:22]) ... ping(wrcl_vip[10.3.219.130:22]) ... ping(hos1[10.3.219.31:22]) ... ping(wrco4[10.3.219.135:22]) ... ping(wrco2[10.3.219.179:22]) ... 

wrco1[10.3.219.178:22],OK
wrcl_vip[10.3.219.130:22],OK
wrco0[10.3.219.177:22],OK
hos2[10.3.219.32:22],OK
hlm1vm[10.3.219.30:22],OK
hos1[10.3.219.31:22],OK
wrcl1[10.3.219.131:22],OK
wrco4[10.3.219.135:22],OK
wrco6[10.3.219.137:22],OK
hlm1[10.3.216.10:22],OK
wrco3[10.3.219.134:22],OK


#### Show Endpoint URLs status (check ports open)

In [104]:
display_html_endpoint_urls(conn)

10.3.216.30:9292 [image],OK
10.3.216.30:5000 [identity],OK
10.3.216.130:9696 [network],OK
10.3.216.30:8776 [volume],OK
10.3.216.130:8774 [compute],OK


In [151]:
#print(inventory)

import paramiko

#hosts = list(inventory['ssh_check'].keys()).sorted()
ssh_hosts = sorted(inventory['ssh_check'])

for host in ssh_hosts:    
    ip = inventory['hosts'][host]['ansible_host']
    user = inventory['hosts'][host]['ansible_user']
    pkey = inventory['hosts'][host]['ssh_key']
    
    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    ssh.connect(ip, username=user, key_filename=pkey, look_for_keys=False)
    stdin, stdout, stderr = ssh.exec_command("uptime")
    
    line=str(stdout.read())
    #print(line)
    up_pos = line.find(" up ")
    if up_pos != -1:
        up_str = line[up_pos + 4:]
        comma_pos = up_str.find(",")
        if comma_pos != -1:
            comma_pos2 = up_str[comma_pos+1:].find(",")
            if comma_pos2 != -1:
                up_str=up_str[:comma_pos+comma_pos2+1]
                print(host + ":" + up_str)
    
#inventory

hlm1:99 days,  5:30
hlm1vm:97 days, 20:50
hos1:84 days, 23:54
hos2:84 days, 23:54
hos3:29 days,  2:06
hos_vip:84 days, 23:54
wrcl1:48 days, 28 min
wrcl2:57 days,  6:29
wrcl_vip:57 days,  6:29
wrco0:57 days,  3:55
wrco1:46 days, 23:58
wrco2:47 days,  3:13
wrco3:47 days,  3:09
wrco4:13 days, 5 min
wrco5:13 days, 5 min
wrco6:10 days, 19:44
wrco7:10 days, 19:44


In [166]:

df_hosts = sorted(inventory['df_check'])

for host in df_hosts:    
    ip = inventory['hosts'][host]['ansible_host']
    user = inventory['hosts'][host]['ansible_user']
    pkey = inventory['hosts'][host]['ssh_key']
    df_check = inventory['hosts'][host]['df_check']
    #print(df_check)
    
    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    ssh.connect(ip, username=user, key_filename=pkey, look_for_keys=False)
    #stdin, stdout, stderr = ssh.exec_command("df -n+2 " + df_check.replace(",", " "))
    df_cmd="df " + df_check.replace(",", " ") + "| grep -v ^Filesystem"
    stdin, stdout, stderr = ssh.exec_command(df_cmd)
    df_op = stdout.read().decode('utf8')
    #print("HOST[" + host + "]<" + df_check + ">{" + df_cmd +"}:" + df_op)
    
    df_lines=df_op.split("\n")
    for df_line in df_lines:
        #print("LINE: " + df_line)
        pc_pos = df_line.find("%")
        if pc_pos != -1:
            pc=int(df_line[pc_pos-3:pc_pos])
            partn=df_line[pc_pos+1:]
            print(host + " " + str(pc) + "% " + partn)


hlm1 21%  /
hlm1vm 4%  /
hos1 52%  /
hos2 52%  /
hos3 53%  /
wrcl1 23%  /
wrcl1 1%  /scratch
wrcl1 11%  /var/log
wrcl1 1%  /opt/backups
wrcl1 23%  /
wrcl1 23%  /
wrcl1 23%  /
wrcl1 23%  /
wrcl2 22%  /
wrcl2 4%  /scratch
wrcl2 13%  /var/log
wrcl2 1%  /opt/backups
wrcl2 2%  /var/lib/rabbitmq
wrcl2 4%  /var/lib/postgresql
wrcl2 25%  /opt/cgcs
wrcl2 1%  /opt/platform
wrco0 11%  /
wrco0 3%  /var/log
wrco0 1%  /scratch
wrco0 1%  /local_pv
wrco0 25%  /etc/nova/instances
wrco0 1%  /opt/platform
wrco1 11%  /
wrco1 3%  /var/log
wrco1 1%  /scratch
wrco1 1%  /local_pv
wrco1 25%  /etc/nova/instances
wrco1 1%  /opt/platform
wrco2 11%  /
wrco2 5%  /var/log
wrco2 1%  /scratch
wrco2 1%  /local_pv
wrco2 25%  /etc/nova/instances
wrco2 1%  /opt/platform
wrco3 11%  /
wrco3 3%  /var/log
wrco3 1%  /scratch
wrco3 1%  /local_pv
wrco3 25%  /etc/nova/instances
wrco3 1%  /opt/platform
wrco4 11%  /
wrco4 3%  /var/log
wrco4 1%  /scratch
wrco4 1%  /local_pv
wrco4 25%  /etc/nova/instances
wrco4 1%  /opt/platform
wrco

In [167]:
# TODO: 
# equivalent of openstack service list